In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./aave_barrow_contract.csv')
df = data.copy()

In [6]:
import pandas as pd

# Read the dataset into a pandas DataFrame
#df = pd.read_csv("your_dataset.csv")

# Define the scoring criteria and intervals
deposit_value_intervals = [0, 1, 10, 100, 1000, 10000]
deposit_value_scores = [10, 25, 40, 55, 70, 100]

deposit_time_intervals = [0, 30, 90, 180, 365]
deposit_time_scores = [10, 30, 50, 70, 100]

borrow_value_intervals = [0, 1, 10, 100, 1000, 10000]
borrow_value_scores = [10, 25, 40, 55, 70, 100]

borrow_time_intervals = [0, 30, 90, 180, 365]
borrow_time_scores = [10, 30, 50, 70, 100]

repayment_value_intervals = [0, 1, 10, 100, 1000, 10000]
repayment_value_scores = [10, 25, 40, 55, 70, 100]

# Create empty dataframes for each category
deposit_df = pd.DataFrame(columns=['Address', 'Value_IN(ETH)', 'Deposit Value Score'])
deposit_time_df = pd.DataFrame(columns=['Address', 'DateTime', 'Deposit Time Score'])
borrow_df = pd.DataFrame(columns=['Address', 'Value_OUT(ETH)', 'Borrow Value Score'])
borrow_time_df = pd.DataFrame(columns=['Address', 'DateTime', 'Borrow Time Score'])
repayment_df = pd.DataFrame(columns=['Address', 'Value_IN(ETH)', 'Repayment Value Score'])

# Iterate over each unique address in the dataset
for address, group_data in df.groupby('From'):
    method = group_data['Method'].unique()[0]
    
    if 'Deposit ETH' in method:
        value_in_eth = group_data['Value_IN(ETH)'].sum()
        deposit_value_score = None
        deposit_time_score = None
        
        # Calculate the score for deposit value
        if 0 <= value_in_eth < 1:
            deposit_value_score = 10
        elif 1 <= value_in_eth < 10:
            deposit_value_score = 25
        elif 10 <= value_in_eth < 100:
            deposit_value_score = 40
        elif 100 <= value_in_eth < 1000:
            deposit_value_score = 55
        elif 1000 <= value_in_eth < 10000:
            deposit_value_score = 70
        else:
            deposit_value_score = 100
        
        deposit_df = pd.concat([deposit_df, pd.DataFrame({'Address': [address], 'Value_IN(ETH)': [value_in_eth], 'Deposit Value Score': [deposit_value_score]})], ignore_index=True)
        
        # Calculate the score for deposit time
        days_since_deposit = (pd.Timestamp.now() - pd.to_datetime(group_data['DateTime'])).dt.days
        if days_since_deposit.iloc[0] <= 30:
            deposit_time_score = 10
        elif days_since_deposit.iloc[0] <= 90:
            deposit_time_score = 30
        elif days_since_deposit.iloc[0] <= 180:
            deposit_time_score = 50
        elif days_since_deposit.iloc[0] <= 365:
            deposit_time_score = 70
        else:
            deposit_time_score = 100
        
        deposit_time_df = pd.concat([deposit_time_df, pd.DataFrame({'Address': [address], 'DateTime': [group_data['DateTime'].iloc[0]], 'Deposit Time Score': [deposit_time_score]})], ignore_index=True)

    elif 'Borrow ETH' in method:
        value_out_eth = group_data['Value_OUT(ETH)'].sum()
        borrow_value_score = None
        borrow_time_score = None
        
        # Calculate the score for borrow value
        if 0 <= value_out_eth < 1:
            borrow_value_score = 10
        elif 1 <= value_out_eth < 10:
            borrow_value_score = 25
        elif 10 <= value_out_eth < 100:
            borrow_value_score = 40
        elif 100 <= value_out_eth < 1000:
            borrow_value_score = 55
        elif 1000 <= value_out_eth < 10000:
            borrow_value_score = 70
        else:
            borrow_value_score = 100
        
        borrow_df = pd.concat([borrow_df, pd.DataFrame({'Address': [address], 'Value_OUT(ETH)': [value_out_eth], 'Borrow Value Score': [borrow_value_score]})], ignore_index=True)
        
        # Calculate the score for borrow time
        days_since_borrow = (pd.Timestamp.now() - pd.to_datetime(group_data['DateTime'])).dt.days
        if days_since_borrow.iloc[0] <= 30:
            borrow_time_score = 10
        elif days_since_borrow.iloc[0] <= 90:
            borrow_time_score = 30
        elif days_since_borrow.iloc[0] <= 180:
            borrow_time_score = 50
        elif days_since_borrow.iloc[0] <= 365:
            borrow_time_score = 70
        else:
            borrow_time_score = 100
        
        borrow_time_df = pd.concat([borrow_time_df, pd.DataFrame({'Address': [address], 'DateTime': [group_data['DateTime'].iloc[0]], 'Borrow Time Score': [borrow_time_score]})], ignore_index=True)

    elif 'Repay ETH' in method:
        value_in_eth = group_data['Value_IN(ETH)'].sum()
        repayment_value_score = None
        
        # Calculate the score for repayment value
        if 0 <= value_in_eth < 1:
            repayment_value_score = 10
        elif 1 <= value_in_eth < 10:
            repayment_value_score = 25
        elif 10 <= value_in_eth < 100:
            repayment_value_score = 40
        elif 100 <= value_in_eth < 1000:
            repayment_value_score = 55
        elif 1000 <= value_in_eth < 10000:
            repayment_value_score = 70
        else:
            repayment_value_score = 100
        
        repayment_df = pd.concat([repayment_df, pd.DataFrame({'Address': [address], 'Value_IN(ETH)': [value_in_eth], 'Repayment Value Score': [repayment_value_score]})], ignore_index=True)

# Merge all dataframes based on the 'Address' column
lp_scores_df = pd.merge(deposit_df, deposit_time_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, borrow_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, borrow_time_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, repayment_df, on='Address', how='outer')

# Calculate LP scores for each unique address by summing up all the scores
lp_scores_df['LP Score'] = lp_scores_df[['Deposit Value Score', 'Deposit Time Score', 'Borrow Value Score', 'Borrow Time Score', 'Repayment Value Score']].sum(axis=1)

# Calculate LP rewards for each unique address
lp_scores_df['LP Reward'] = (lp_scores_df['LP Score'] / 500) * 100

# Print the merged dataframe
print("Merged Dataframe:")
print(lp_scores_df)


Merged Dataframe:
                                         Address  Value_IN(ETH)_x  \
0     0x0000000a5d2afc64886a80780bc5ca6ecc519181         0.000005   
1     0x0000000f3a9e446d0a6f976df5f0ed1b324dcd0e         0.000005   
2     0x00002b503a75998c97508916a74fdb41934fa030        64.800000   
3     0x0000ce08fa224696a819877070bf378e8b131acf         0.001000   
4     0x0026c9a5bfbee977ad03dcb4a95cfa715278e779         0.012160   
...                                          ...              ...   
1855  0xfc1d987a42a0e37bd19ae7bc8d7c0566b8555d9c              NaN   
1856  0xfcc1b0aaeb2b9063ab34fdfbb3eedc2ff30ae4f2              NaN   
1857  0xfd8a8ff896ccd28efe1c4a3ef69b9c4fd3f1943d              NaN   
1858  0xffb3468b6861ba7044fcf61a2d976028204efc49              NaN   
1859  0xffc57ab280cb8d1e47e11ce225f113612cff3d8e              NaN   

     Deposit Value Score        DateTime_x Deposit Time Score Value_OUT(ETH)  \
0                     10  09/09/2022 03:35                 70            

In [8]:
lp_scores_df.tail(3)

,Address,Value_IN(ETH)_x,Deposit Value Score,DateTime_x,Deposit Time Score,Value_OUT(ETH),Borrow Value Score,DateTime_y,Borrow Time Score,Value_IN(ETH)_y,Repayment Value Score,LP Score,LP Reward
1857,0xfd8a8ff896ccd28efe1c4a3ef69b9c4fd3f1943d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.704783,10,10.0,2.0
1858,0xffb3468b6861ba7044fcf61a2d976028204efc49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.183037,25,25.0,5.0
1859,0xffc57ab280cb8d1e47e11ce225f113612cff3d8e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001009,10,10.0,2.0


In [12]:
lp_scores_df.sort_values(by='LP Score', ascending=False).head(3)

,Address,Value_IN(ETH)_x,Deposit Value Score,DateTime_x,Deposit Time Score,Value_OUT(ETH),Borrow Value Score,DateTime_y,Borrow Time Score,Value_IN(ETH)_y,Repayment Value Score,LP Score,LP Reward
754,0x6e2e4991ebc00841e10419065c966b613bc4a84b,407.198811,55,06/09/2022 17:17,100,NaN,NaN,NaN,NaN,NaN,NaN,155.0,31.0
598,0x58a24fa9ae8847cbcf245dd2ef7fcef205927af1,243.428086,55,07/09/2022 13:07,100,NaN,NaN,NaN,NaN,NaN,NaN,155.0,31.0
761,0x6fb6853dc9a36d98d415f9c68c7d2fc0c55198e6,40.343433,40,07/09/2022 17:25,100,NaN,NaN,NaN,NaN,NaN,NaN,140.0,28.0


In [21]:


data[data['From']=='0x00002b503a75998c97508916a74fdb41934fa030']

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,ContractAddress,Value_IN(ETH),Value_OUT(ETH),CurrentValue @ $1869.43/Eth,TxnFee(ETH),TxnFee(USD),Historical $Price/Eth,Method
3489,0xec92abd694c4308623bc5ba17a1dd858524b39698474...,23241169,1663230048,15/09/2022 08:20,0x00002b503a75998c97508916a74fdb41934fa030,0x76d3030728e52deb8848d5613abade88441cbc59,NaN,64.8,0,121139.064,0.00012,0.22476,1472.51,Deposit ETH


In [15]:
import pandas as pd

# Read the dataset into a pandas DataFrame
#df = pd.read_csv("your_dataset.csv")

# Define the scoring criteria and intervals
deposit_value_intervals = [0, 1, 10, 100, 1000, 10000]
deposit_value_scores = [10, 25, 40, 55, 70, 100]

deposit_time_intervals = [0, 30, 90, 180, 365]
deposit_time_scores = [10, 30, 50, 70, 100]

borrow_value_intervals = [0, 1, 10, 100, 1000, 10000]
borrow_value_scores = [10, 25, 40, 55, 70, 100]

borrow_time_intervals = [0, 30, 90, 180, 365]
borrow_time_scores = [10, 30, 50, 70, 100]

repayment_value_intervals = [0, 1, 10, 100, 1000, 10000]
repayment_value_scores = [10, 25, 40, 55, 70, 100]

# Create empty dataframes for each category
deposit_df = pd.DataFrame(columns=['Address', 'Value_IN(ETH)', 'Deposit Value Score'])
deposit_time_df = pd.DataFrame(columns=['Address', 'DateTime', 'Deposit Time Score'])
borrow_df = pd.DataFrame(columns=['Address', 'Value_OUT(ETH)', 'Borrow Value Score'])
borrow_time_df = pd.DataFrame(columns=['Address', 'DateTime', 'Borrow Time Score'])
repayment_df = pd.DataFrame(columns=['Address', 'Value_IN(ETH)', 'Repayment Value Score'])

# Iterate over each unique address in the dataset
for address, group_data in df.groupby('From'):
    methods = group_data['Method'].unique()
    for method in methods:
        if 'Deposit ETH' in method:
            value_in_eth = group_data['Value_IN(ETH)'].sum()
            deposit_value_score = None
            deposit_time_score = None

            # Calculate the score for deposit value
            if 0 <= value_in_eth < 1:
                deposit_value_score = 10
            elif 1 <= value_in_eth < 10:
                deposit_value_score = 25
            elif 10 <= value_in_eth < 100:
                deposit_value_score = 40
            elif 100 <= value_in_eth < 1000:
                deposit_value_score = 55
            elif 1000 <= value_in_eth < 10000:
                deposit_value_score = 70
            else:
                deposit_value_score = 100

            deposit_df = pd.concat([deposit_df, pd.DataFrame({'Address': [address], 'Value_IN(ETH)': [value_in_eth], 'Deposit Value Score': [deposit_value_score]})], ignore_index=True)

            # Calculate the score for deposit time
            days_since_deposit = (pd.Timestamp.now() - pd.to_datetime(group_data['DateTime'])).dt.days
            if days_since_deposit.iloc[0] <= 30:
                deposit_time_score = 10
            elif days_since_deposit.iloc[0] <= 90:
                deposit_time_score = 30
            elif days_since_deposit.iloc[0] <= 180:
                deposit_time_score = 50
            elif days_since_deposit.iloc[0] <= 365:
                deposit_time_score = 70
            else:
                deposit_time_score = 100

            deposit_time_df = pd.concat([deposit_time_df, pd.DataFrame({'Address': [address], 'DateTime': [group_data['DateTime'].iloc[0]], 'Deposit Time Score': [deposit_time_score]})], ignore_index=True)

        elif 'Borrow ETH' in method:
            value_out_eth = group_data['Value_OUT(ETH)'].sum()
            borrow_value_score = None
            borrow_time_score = None

            # Calculate the score for borrow value
            if 0 <= value_out_eth < 1:
                borrow_value_score = 10
            elif 1 <= value_out_eth < 10:
                borrow_value_score = 25
            elif 10 <= value_out_eth < 100:
                borrow_value_score = 40
            elif 100 <= value_out_eth < 1000:
                borrow_value_score = 55
            elif 1000 <= value_out_eth < 10000:
                borrow_value_score = 70
            else:
                borrow_value_score = 100

            borrow_df = pd.concat([borrow_df, pd.DataFrame({'Address': [address], 'Value_OUT(ETH)': [value_out_eth], 'Borrow Value Score': [borrow_value_score]})], ignore_index=True)

            # Calculate the score for borrow time
            days_since_borrow = (pd.Timestamp.now() - pd.to_datetime(group_data['DateTime'])).dt.days
            if days_since_borrow.iloc[0] <= 30:
                borrow_time_score = 10
            elif days_since_borrow.iloc[0] <= 90:
                borrow_time_score = 30
            elif days_since_borrow.iloc[0] <= 180:
                borrow_time_score = 50
            elif days_since_borrow.iloc[0] <= 365:
                borrow_time_score = 70
            else:
                borrow_time_score = 100

            borrow_time_df = pd.concat([borrow_time_df, pd.DataFrame({'Address': [address], 'DateTime': [group_data['DateTime'].iloc[0]], 'Borrow Time Score': [borrow_time_score]})], ignore_index=True)

        elif 'Repay ETH' in method:
            value_in_eth = group_data['Value_IN(ETH)'].sum()
            repayment_value_score = None

            # Calculate the score for repayment value
            if 0 <= value_in_eth < 1:
                repayment_value_score = 10
            elif 1 <= value_in_eth < 10:
                repayment_value_score = 25
            elif 10 <= value_in_eth < 100:
                repayment_value_score = 40
            elif 100 <= value_in_eth < 1000:
                repayment_value_score = 55
            elif 1000 <= value_in_eth < 10000:
                repayment_value_score = 70
            else:
                repayment_value_score = 100

            repayment_df = pd.concat([repayment_df, pd.DataFrame({'Address': [address], 'Value_IN(ETH)': [value_in_eth], 'Repayment Value Score': [repayment_value_score]})], ignore_index=True)

# Merge all dataframes based on the 'Address' column
lp_scores_df = pd.merge(deposit_df, deposit_time_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, borrow_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, borrow_time_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, repayment_df, on='Address', how='outer')

# Calculate LP scores for each unique address by summing up all the scores
lp_scores_df['LP Score'] = lp_scores_df[['Deposit Value Score', 'Deposit Time Score', 'Borrow Value Score', 'Borrow Time Score', 'Repayment Value Score']].sum(axis=1)

# Calculate LP rewards for each unique address
lp_scores_df['LP Reward'] = (lp_scores_df['LP Score'] / 500) * 100

# Print the merged dataframe
print("Merged Dataframe:")
print(lp_scores_df)


Merged Dataframe:
                                         Address  Value_IN(ETH)_x  \
0     0x0000000a5d2afc64886a80780bc5ca6ecc519181         0.000005   
1     0x0000000f3a9e446d0a6f976df5f0ed1b324dcd0e         0.000005   
2     0x00002b503a75998c97508916a74fdb41934fa030        64.800000   
3     0x0000ce08fa224696a819877070bf378e8b131acf         0.001000   
4     0x0026c9a5bfbee977ad03dcb4a95cfa715278e779         0.012160   
...                                          ...              ...   
1983  0xf576dae6f10ea36a4dfade275aa576560e3b4b4a              NaN   
1984  0xfbd08fd2b2c7148b4e38624158c3155b5b49f886              NaN   
1985  0xfc1d987a42a0e37bd19ae7bc8d7c0566b8555d9c              NaN   
1986  0xfd8a8ff896ccd28efe1c4a3ef69b9c4fd3f1943d              NaN   
1987  0xffc57ab280cb8d1e47e11ce225f113612cff3d8e              NaN   

     Deposit Value Score        DateTime_x Deposit Time Score Value_OUT(ETH)  \
0                     10  09/09/2022 03:35                 70            

In [16]:
lp_scores_df[lp_scores_df['Address']=='0x6e2e4991ebc00841e10419065c966b613bc4a84b']

,Address,Value_IN(ETH)_x,Deposit Value Score,DateTime_x,Deposit Time Score,Value_OUT(ETH),Borrow Value Score,DateTime_y,Borrow Time Score,Value_IN(ETH)_y,Repayment Value Score,LP Score,LP Reward
811,0x6e2e4991ebc00841e10419065c966b613bc4a84b,407.198811,55,06/09/2022 17:17,100,0,10,06/09/2022 17:17,100,407.198811,55,320.0,64.0


In [17]:
lp_scores_df = lp_scores_df.drop(columns=['DateTime_x', 'DateTime_y'])

lp_scores_df.head(3)

,Address,Value_IN(ETH)_x,Deposit Value Score,Deposit Time Score,Value_OUT(ETH),Borrow Value Score,Borrow Time Score,Value_IN(ETH)_y,Repayment Value Score,LP Score,LP Reward
0,0x0000000a5d2afc64886a80780bc5ca6ecc519181,0.000005,10,70,NaN,NaN,NaN,NaN,NaN,80.0,16.0
1,0x0000000f3a9e446d0a6f976df5f0ed1b324dcd0e,0.000005,10,70,NaN,NaN,NaN,NaN,NaN,80.0,16.0
2,0x00002b503a75998c97508916a74fdb41934fa030,64.800000,40,70,NaN,NaN,NaN,NaN,NaN,110.0,22.0


In [18]:
lp_scores_df['Value_IN(ETH)_y'].sum()

29440.064883206473

In [19]:
# Calculate the new column 'Value_IN(ETH)'
lp_scores_df['Value_IN(ETH)'] = lp_scores_df['Value_IN(ETH)_x'] + lp_scores_df['Value_IN(ETH)_y']

# Drop the unwanted columns
lp_scores_df = lp_scores_df.drop(columns=['Value_IN(ETH)_x', 'Value_IN(ETH)_y'])

# Rearrange the columns
lp_scores_df = lp_scores_df[['Address', 'Value_IN(ETH)', 'Value_OUT(ETH)', 'Deposit Value Score', 'Deposit Time Score', 'Borrow Value Score', 'Borrow Time Score', 'Repayment Value Score', 'LP Score', 'LP Reward']]

# Print the updated dataframe
lp_scores_df.head(3)


,Address,Value_IN(ETH),Value_OUT(ETH),Deposit Value Score,Deposit Time Score,Borrow Value Score,Borrow Time Score,Repayment Value Score,LP Score,LP Reward
0,0x0000000a5d2afc64886a80780bc5ca6ecc519181,NaN,NaN,10,70,NaN,NaN,NaN,80.0,16.0
1,0x0000000f3a9e446d0a6f976df5f0ed1b324dcd0e,NaN,NaN,10,70,NaN,NaN,NaN,80.0,16.0
2,0x00002b503a75998c97508916a74fdb41934fa030,NaN,NaN,40,70,NaN,NaN,NaN,110.0,22.0


In [ ]:
lp_scor = lp_scores_df.fillna(0)